In [ ]:
from bs4 import BeautifulSoup as soup
from splinter import Browser
from pprint import pprint
import time

# Visit and Parse the first page of Ebay to test

In [ ]:
browser=Browser('chrome')

# Visit the URL
url = "https://www.ebay.com/b/Cars-Trucks/6001?mag=1&_fsrp=0&rt=nc&_sacat=6001&Model%2520Year=2010%7C2011%7C2012%7C2013%7C2014%7C2015%7C2016%7C2017%7C2018%7C2019%7C2020&LH_ItemCondition=3000%7C1000%7C2500&LH_Sold=1&LH_PrefLoc=1"
browser.visit(url)

# Parse the HTML
html = browser.html
mySoup = soup(html, 'html.parser')

In [ ]:
elements = mySoup.find_all("li", class_="s-item")
pprint(elements)

In [ ]:
# Convert the total results string into an int
total_results = mySoup.find("h2", class_="srp-controls__count-heading")
total_results_text = total_results.text
total_results_text = total_results_text[0:5]
total_results_text = total_results_text.replace(',', '')
total_results_int = int(total_results_text)

print(total_results_int)

## Method to scrape the page the browser is on

In [ ]:
def scrape_page():
    my_list = []
    
    html = browser.html
    page = soup(html, 'html.parser')
    
    list_elements = page.find_all("li", class_="s-item")
    
    for element in list_elements:
        title = element.find(class_="s-item__title")
        price = element.find(class_="s-item__price")
        sold_date = element.find(class_="s-item__sponsor")
        item_url = element.find("a")
        
        dict = {
            "title" : title.text,
            "price" : price.text,
            "sold_date" : sold_date.text,
            "item_url" : item_url['href']
        }
        
        my_list.append(dict)
    
    
    return my_list

# Scraping Loop

In [ ]:
vehicles = []
page_num = 1

# for _ in range(5):
while len(vehicles) <= total_results_int:
    page_url = f"https://www.ebay.com/b/Cars-Trucks/6001?LH_ItemCondition=3000%7C1000%7C2500&LH_PrefLoc=1&LH_Sold=1&Model%2520Year=2010%7C2011%7C2012%7C2013%7C2014%7C2015%7C2016%7C2017%7C2018%7C2019%7C2020&mag=1&rt=nc&_fsrp=0&_pgn={page_num}&_sacat=6001"
    browser.visit(page_url)
    
    vehicles.extend(scrape_page())
    
    page_num += 1
    
    time.sleep(8)
    print(len(vehicles))
    
pprint(vehicles)
print(len(vehicles))

## Create a json object out of the data that was scraped and save it to 'vehicles.json'

In [ ]:
import json

json_object = json.dumps(vehicles, indent = 4)

with open('data/vehicles.json', 'w') as outfile:
    json.dump(vehicles, outfile)

In [ ]:
browser.quit()